In [ ]:
strsrtfilename = "Mass 2021 1080p.sample.srt"
output_filename = "Mass 2021 1080p.sample French.srt"
#strsrtfilename = "Mass 2021 1080p.srt"
#output_filename = "Mass 2021 1080p.French.srt"

In [49]:
with open(strsrtfilename, 'rb') as file:
    srt_text = file.read()
if b'\r\n' in srt_text:
    print("Windows-style newline: '\\r\\n'")
elif b'\n' in srt_text:
    print("Unix/Linux-style newline: '\\n'")
elif b'\r' in srt_text:
    print("Old Mac-style newline: '\\r'")
else:
    print("No newline sequence found.")

Windows-style newline: '\r\n'


In [50]:
from langchain_openai import ChatOpenAI
#from langchain_text_splitters import TokenTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableParallel
import getpass

In [51]:
with open(strsrtfilename, 'r', encoding='utf-8') as file:
    srt_text = file.read()

In [52]:
OPENAI_API_KEY = getpass.getpass('Enter your OPENAI_API_KEY')

In [53]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY,model_name="gpt-4o-mini")

In [54]:
# Split
text_chunks_chain = (
    RunnableLambda(lambda x: 
        [
            {
                'chunk': text_chunk, 
            }
            for text_chunk in 
               CharacterTextSplitter(separator="\n\n", chunk_size=20, chunk_overlap=0, length_function=len, is_separator_regex=False).split_text(x)
        ]
    )
)

In [55]:
# Map
translate_chunk_prompt_template = """You are an experienced translater to French. 
Translate the following subtitle text to French and leave the two first lines unchanged. 
Do not add any comment to the answer, only the translated text. 
Also remove the original English text in the result. 
Text: {chunk}"""
translate_chunk_prompt = PromptTemplate.from_template(translate_chunk_prompt_template)
translate_chunk_chain = translate_chunk_prompt | llm

translate_map_chain = (
    RunnableParallel (
        {
            'translation': translate_chunk_chain | StrOutputParser()        
        }
    )
)

In [56]:
# Reduce
translate_reduce_chain = (
    RunnableLambda(lambda x: '\n\n'.join([i['translation'] for i in x]))
)

In [57]:
map_reduce_chain = (
   text_chunks_chain
   | translate_map_chain.map()
   | translate_reduce_chain
)

In [58]:
translation = map_reduce_chain.invoke(srt_text)

Created a chunk of size 50, which is longer than the specified 20
Created a chunk of size 36, which is longer than the specified 20
Created a chunk of size 48, which is longer than the specified 20
Created a chunk of size 35, which is longer than the specified 20
Created a chunk of size 51, which is longer than the specified 20
Created a chunk of size 43, which is longer than the specified 20
Created a chunk of size 44, which is longer than the specified 20
Created a chunk of size 63, which is longer than the specified 20
Created a chunk of size 82, which is longer than the specified 20
Created a chunk of size 88, which is longer than the specified 20
Created a chunk of size 56, which is longer than the specified 20
Created a chunk of size 85, which is longer than the specified 20
Created a chunk of size 52, which is longer than the specified 20
Created a chunk of size 40, which is longer than the specified 20
Created a chunk of size 73, which is longer than the specified 20
Created a 

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-wAXzG4TDwxNJRkpcAgjJjcEJ on requests per min (RPM): Limit 500, Used 500, Requested 1. Please try again in 120ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
print(translation)

1  
00:02:30,411 --> 00:02:31,977  
Je suis en retard, je sais.

2  
00:02:35,764 --> 00:02:36,852  
Hein.

3  
00:02:36,895 --> 00:02:38,723  
Quelqu'un a-t-il appelé ?

4  
00:02:38,767 --> 00:02:40,203  
Non.

5  
00:02:40,247 --> 00:02:41,639  
Kendra n'a pas appelé ?

6  
00:02:41,683 --> 00:02:43,250  
Non. Personne.

7  
00:02:46,949 --> 00:02:48,168  
C'est bien.

8  
00:02:52,346 --> 00:02:54,696  
Mon fils avait un match ce matin,

9  
00:02:54,739 --> 00:02:56,500  
et il voulait que j'attende  
son père pour qu'il arrive,

10  
00:02:56,524 --> 00:03:00,397  
et puis je suis tombé dans  
des embouteillages ou un accident...  


In [ ]:
# Save translation to a file
with open(output_filename, 'w', encoding='utf-8') as f:
    f.write(translation)

print(f"Translation saved to {output_filename}")

Translation saved to Mass_2021_1080p_sample_FR.srt
